Machine Learnig sistema de recomendaciones

In [1]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors

In [4]:
df_movies = pd.read_csv('C:/Users/cesua/OneDrive/Documentos/Henry/Proyecto_Individual_1/Datasets/movies_dataset_final.csv')

C:\Users\cesua\AppData\Local\Temp\ipykernel_17888\2566478040.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movies = pd.read_csv('C:/Users/cesua/OneDrive/Documentos/Henry/Proyecto_Individual_1/Datasets/movies_dataset_final.csv')


In [5]:
df_movies.head()

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,release_date,revenue,...,status,tagline,title,vote_average,id_btc,name_btc,genre_names,production_countries,return,release_year
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,Pixar Animation Studios,1995-10-30,373554033.0,...,Released,NaN,Toy Story,7.7,10194.0,Toy Story Collection,"Animation, Comedy, Family",United States of America,12.45,1995.0
1,NaN,65000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"TriStar Pictures, Teitler Film, Interscope Com...",1995-12-15,262797249.0,...,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,NaN,NaN,"Adventure, Fantasy, Family",United States of America,4.04,1995.0
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,A family wedding reignites the ancient feud be...,11.7129,"Warner Bros., Lancaster Gate",1995-12-22,0.0,...,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,119050.0,Grumpy Old Men Collection,"Romance, Comedy",United States of America,0.00,1995.0
3,NaN,16000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,Twentieth Century Fox Film Corporation,1995-12-22,81452156.0,...,Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,NaN,NaN,"Comedy, Drama, Romance",United States of America,5.09,1995.0
4,"{'id': 96871, 'name': 'Father of the Bride Col...",0.0,"[{'id': 35, 'name': 'Comedy'}]",11862,en,Just when George Banks has recovered from his ...,8.387519,"Sandollar Productions, Touchstone Pictures",1995-02-10,76578911.0,...,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,96871.0,Father of the Bride Collection,Comedy,United States of America,0.00,1995.0


In [6]:
df_movies.info(
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45379 entries, 0 to 45378
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  4491 non-null   object 
 1   budget                 45376 non-null  float64
 2   genres                 45379 non-null  object 
 3   id                     45379 non-null  object 
 4   original_language      45368 non-null  object 
 5   overview               44438 non-null  object 
 6   popularity             45377 non-null  object 
 7   production_companies   33522 non-null  object 
 8   release_date           45376 non-null  object 
 9   revenue                45379 non-null  float64
 10  runtime                45130 non-null  float64
 11  spoken_languages       45376 non-null  object 
 12  status                 45296 non-null  object 
 13  tagline                20398 non-null  object 
 14  title                  45376 non-null  object 
 15  vo

In [16]:
# Ordenar la tabla según las columnas específicas
columns_order = ['id', 'title', 'genre_names', 'popularity']
df_movies = df_movies[columns_order]

df_movies['genre_names'] = df_movies['genre_names'].str.replace(r'[\'"\s\[\]{}\\]', ' ', regex=True)
# Eliminar las comas de la columna "genero"
df_movies['genre_names'] = df_movies['genre_names'].str.replace(',', '')

In [17]:
df_movies['popularity'] = pd.to_numeric(df_movies['popularity'], errors='coerce')

In [18]:
# Redondear la columna "popularity" a dos decimales
df_movies['popularity'] = df_movies['popularity'].round(2)

In [19]:
df_movies

,id,title,genre_names,popularity
0,862,Toy Story,Animation Comedy Family,21.95
1,8844,Jumanji,Adventure Fantasy Family,17.02
2,15602,Grumpier Old Men,Romance Comedy,11.71
3,31357,Waiting to Exhale,Comedy Drama Romance,3.86
4,11862,Father of the Bride Part II,Comedy,8.39
...,...,...,...,...
45374,30840,Robin Hood,Drama Action Romance,5.68
45375,111109,Century of Birthing,Drama,0.18
45376,67758,Betrayal,Action Drama Thriller,0.90
45377,227506,Satan Triumphant,NaN,0.00


In [39]:
def movie_recommendation(movie_title):

    # Buscar la película por título en la columna 'title'
    movie = df_movies[df_movies['title'] == movie_title]

    if len(movie) == 0:
        return "La película no se encuentra en la base de datos."

    # Obtener el género y la popularidad de la película
    movie_genre = movie['genre_names'].values[0]
    movie_popularity = movie['popularity'].values[0]

    # Crear una matriz de características para el modelo de vecinos más cercanos
    features = df_movies[['popularity']]
    genres = df_movies['genre_names'].str.get_dummies(sep=' ')
    features = pd.concat([features, genres], axis=1)

    # Manejar valores faltantes (NaN) reemplazándolos por ceros
    features = features.fillna(0)

    # Crear el modelo de vecinos más cercanos
    nn_model = NearestNeighbors(n_neighbors=6, metric='euclidean')
    nn_model.fit(features)

    # Encontrar las películas más similares
    _, indices = nn_model.kneighbors([[movie_popularity] + [0] * len(genres.columns)], n_neighbors=6)

    # Obtener los títulos de las películas recomendadas
    recommendations = df_movies.iloc[indices[0][1:]]['title']

    return recommendations

In [40]:
# Ejemplo de uso de la función
movie_title = 'Toy Story'
recommended_movies = movie_recommendation(movie_title)
print(f"Películas recomendadas para '{movie_title}':")
print(recommended_movies)

Películas recomendadas para 'Toy Story':
17127        The Hangover Part II
13193                Seven Pounds
41859                  Fist Fight
11133    The Pursuit of Happyness
2599                 Ghostbusters
Name: title, dtype: object


c:\Users\cesua\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


In [41]:
df_movies.to_csv('ML_movies.csv', index=False)